In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd

import random
import math
import time


In [ ]:
from datetime import datetime
import datetime
from tqdm.notebook import tqdm

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from torch.utils.data import TensorDataset, DataLoader
from matplotlib import pyplot as plt 

In [ ]:
use_gpu = torch.cuda.is_available()
use_gpu

## MPS model


In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.device = device
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

        
    def forward(self, query, key, value):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]

        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [ ]:
class FeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.feedforward = FeedforwardLayer(hid_dim, pf_dim,dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [batch size, src len, hid dim]
                
        #self attention
        _src, _ = self.self_attention(src, src, src)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        # feedforward
        _src = self.feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        return src

In [ ]:
class TransformerEncoder(nn.Module):
    # 输入数据的维度为(input_dim, hid_dim)
    # pf_dim 是ff的中间层维度
    def __init__(self, 
                 input_dim,
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device):
        super().__init__()

        """
        :param input_dim: 输入的数据长度len
        :param output_dim：
        :param hid_dim: 输入的数据维度dim
        :param n_layers: transformer的层数
        :param n_heads: 多头attention的数量
        :param pf_dim: feed forward的中间层维度
        :param dropout:
        :param device
       
        """
        
        self.device = device

        self.layers = nn.ModuleList([TransformerEncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

        
    def forward(self, src):
        
        #src = [batch size, src len, src hidden]  
        
        src = self.dropout(src)        
        
        for layer in self.layers:
            
            src = layer(src)
            
        #src = [batch size, src len, hid dim]
                  
        return src

In [ ]:
class Encoder(nn.Module):
    
    def __init__(self, input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device):
        
        super().__init__()
        
        self.short_term_transformer = TransformerEncoder(input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device)
        
        self.middle_term_transformer = TransformerEncoder(input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device)
        
        self.long_term_transformer = TransformerEncoder(input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device)
        
    
    def forward(self, src):
        
        short_term_encoding = self.short_term_transformer(src)
        
        middle_term_encoding = self.middle_term_transformer(src)
        
        long_term_encoding = self.long_term_transformer(src)
        
        return short_term_encoding, middle_term_encoding, long_term_encoding

In [ ]:
class MultiTask(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device):
        # input_dim20 hid_dim6
        super().__init__()

        self.encoder = Encoder(input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device)
        
        self.att_short = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.att_middle = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.att_long = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
    
        self.nn_short = nn.Sequential(
            nn.Linear(2*input_dim*hid_dim, 30),            
            nn.ReLU(),
            nn.Linear(30, 3),
        )
        
        self.nn_middle = nn.Sequential(
            nn.Linear(2*input_dim*hid_dim, 30),            
            nn.ReLU(),
            nn.Linear(30, 3),
        )
        
        self.nn_long = nn.Sequential(
            nn.Linear(2*input_dim*hid_dim, 30),            
            nn.ReLU(),
            nn.Linear(30, 3),
        )
        
        self.Tanh_short = nn.Tanh()
        self.Tanh_middle = nn.Tanh()
        self.Tanh_long = nn.Tanh()
        
        self.softmax = nn.Softmax(dim=-1)
        
    
    def soft_attention(self, src, tanh):
        src = src.view(len(src), -1)
        src_h = tanh(src)
        src_softmax = self.softmax(src_h)
        
        src_out = src*src_softmax
        
        return src_out       
        
        
    def forward(self, srcA, srcB):

        short_encodingA, middle_encodingA, long_encodingA = self.encoder(srcA)
        short_encodingB, middle_encodingB, long_encodingB = self.encoder(srcB)
        
        short_encoding = torch.cat([short_encodingA, short_encodingB], dim=1) 
        middle_encoding = torch.cat([middle_encodingA, middle_encodingB], dim=1)
        long_encoding = torch.cat([long_encodingA, long_encodingB], dim=1)
        
        encoding = torch.cat([short_encoding, middle_encoding, long_encoding], dim=1)

        
        short_encoding_att = self.att_short(short_encoding, encoding, encoding)        
        middle_encoding_att = self.att_middle(middle_encoding, encoding, encoding)
        long_encoding_att = self.att_long(long_encoding, encoding, encoding)
        

        batch_size = len(short_encoding_att[0])
#         m1: [10240 x 6], m2: [240 x 30] at
        short_score = self.nn_short(short_encoding_att[0].view(batch_size, -1))
        middle_score = self.nn_middle(middle_encoding_att[0].view(batch_size, -1))
        long_score = self.nn_long(long_encoding_att[0].view(batch_size, -1))
        
        return short_score, middle_score, long_score


## Train and test

In [ ]:
f = '/home/ruyao/self_supervised_model/data/encoding_feature.pkl'
data = pd.read_pickle(f)
data = data.reset_index(drop=True)
data

In [ ]:
split_date = (datetime.date(2019,1,1), datetime.date(2020,1,1), datetime.date(2021,1,1), datetime.date(2021,10,1))
enc_train_data = data[(data['dt']>=split_date[0]) & (data['dt']<split_date[1])].reset_index(drop=True)
enc_valid_data = data[(data['dt']>=split_date[1]) & (data['dt']<split_date[2])].reset_index(drop=True)
enc_test_data = data[(data['dt']>=split_date[2]) & (data['dt']<split_date[3])].reset_index(drop=True)
data.shape, enc_train_data.shape, enc_valid_data.shape, enc_test_data.shape

In [ ]:
def build_data_loader(data, batch_size):

    X_A = torch.tensor(data['daily20_features_x']).to(torch.float32)
    X_B = torch.tensor(data['daily20_features_y']).to(torch.float32)

    y1 = torch.tensor(data['corr1_label'])
    y2 = torch.tensor(data['corr5_label'])
    y3 = torch.tensor(data['corr20_label'])

    data_set = TensorDataset(X_A, X_B, y1, y2, y3) 
    data_loader = DataLoader(data_set, batch_size, shuffle=True, drop_last=False)
    
    return data_loader

In [ ]:
enc_test_data

In [ ]:
batch_size = 256

print('start train data..', datetime.datetime.now())
trainDataLoader = build_data_loader(enc_train_data, batch_size)

print('start valid data..', datetime.datetime.now())
validDataLoader = build_data_loader(enc_valid_data, batch_size)

print('start test data..', datetime.datetime.now())
testDataLoader = build_data_loader(enc_test_data, batch_size)
print('dataload end', datetime.datetime.now())

In [ ]:
import torchsnooper
# @torchsnooper.snoop()
def enc_train(model, iterator, optimizer, criterion):
    
    model.train()

    epoch_log = {"loss":[], "accuracy":[]}

    short_log = {"loss":[], "accuracy":[]}
    middle_log = {"loss":[], "accuracy":[]}
    long_log = {"loss":[], "accuracy":[]}

    for i, batch in tqdm(enumerate(iterator)):

        X_A, X_B, y1, y2, y3 = batch[0], batch[1], batch[2], batch[3], batch[4]

        optimizer.zero_grad()

        short_score, middle_score, long_score = model(X_A.to(device), X_B.to(device))   

        shape2 = short_score.shape[1]
        loss_short = criterion(short_score, y1.to(device))   
        loss_middle = criterion(middle_score, y2.to(device)) 
        loss_long = criterion(long_score, y3.to(device)) 

        loss = (loss_short+loss_middle+loss_long)/3

        loss.backward()

        optimizer.step()

        epoch_log["loss"].append(loss.item())
        epoch_log["accuracy"].append(accuracy_score(np.round(torch.cat([short_score, middle_score, long_score]).argmax(dim=1).detach().cpu().numpy()), torch.cat([y1, y2, y3]).detach().cpu().numpy()))


        short_log["loss"].append(loss_short.item())
        short_log["accuracy"].append(accuracy_score(np.round(short_score.argmax(dim=1).detach().cpu().numpy()), y1.detach().cpu().numpy()))

        middle_log["loss"].append(loss_middle.item())
        middle_log["accuracy"].append(accuracy_score(np.round(middle_score.argmax(dim=1).detach().cpu().numpy()), y2.detach().cpu().numpy()))

        long_log["loss"].append(loss_long.item())
        long_log["accuracy"].append(accuracy_score(np.round(long_score.argmax(dim=1).detach().cpu().numpy()), y3.detach().cpu().numpy()))

        
    return epoch_log, short_log, middle_log, long_log

In [ ]:
def enc_eval(model, iterator, criterion):
    
    model.eval()
    
    epoch_log = {"loss":[], "accuracy":[]}
    
    short_log = {"loss":[], "accuracy":[]}
    middle_log = {"loss":[], "accuracy":[]}
    long_log = {"loss":[], "accuracy":[]}
    
    with torch.no_grad():
    
        for i, batch in tqdm(enumerate(iterator)):

            X_A, X_B, y1, y2, y3 = batch[0], batch[1], batch[2], batch[3], batch[4]

            short_score, middle_score, long_score = model(X_A.to(device), X_B.to(device))   

            shape2 = short_score.shape[1]
            loss_short = criterion(short_score, y1.to(device))   
            loss_middle = criterion(middle_score, y2.to(device)) 
            loss_long = criterion(long_score, y3.to(device)) 

            loss = (loss_short+loss_middle+loss_long)/3

            epoch_log["loss"].append(loss.item())
            epoch_log["accuracy"].append(accuracy_score(np.round(torch.cat([short_score, middle_score, long_score]).argmax(dim=1).detach().cpu().numpy()), torch.cat([y1, y2, y3]).detach().cpu().numpy()))


            short_log["loss"].append(loss_short.item())
            short_log["accuracy"].append(accuracy_score(np.round(short_score.argmax(dim=1).detach().cpu().numpy()), y1.detach().cpu().numpy()))

            middle_log["loss"].append(loss_middle.item())
            middle_log["accuracy"].append(accuracy_score(np.round(middle_score.argmax(dim=1).detach().cpu().numpy()), y2.detach().cpu().numpy()))

            long_log["loss"].append(loss_long.item())
            long_log["accuracy"].append(accuracy_score(np.round(long_score.argmax(dim=1).detach().cpu().numpy()), y3.detach().cpu().numpy()))

        
    return epoch_log, short_log, middle_log, long_log

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# 参数修改
encoding_model_save_file = '/home/ruyao/self_supervised_model/model/encoding_lr0.0001_epoch25.pt'
pic_name = '/home/ruyao/self_supervised_model/model/encoding_lr0.0001_epoch25_drop3_sigmoid_2019-2'

# input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device
input_dim = 20
hid_dim = 6
n_layers = 1
n_heads = 1
pf_dim = 30
dropout = 0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

LEARNING_RATE = 0.0001
N_EPOCHS = 25

encoding_model = MultiTask(input_dim, hid_dim, n_layers, n_heads, pf_dim, dropout, device).to(device)
# criterion = torch.nn.BCELoss()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(encoding_model.parameters(), lr = LEARNING_RATE)

In [ ]:
encoding_model

In [ ]:
best_valid_loss = float('inf')
train_log = {'loss':[], "acc":[]}
val_log = {'loss':[], "acc":[]}

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = enc_train(encoding_model, trainDataLoader, optimizer, criterion)
    valid_loss = enc_eval(encoding_model, validDataLoader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)    
    
    tra_loss = np.mean(train_loss[0]['loss'])
    tra_acc = np.mean(train_loss[0]['accuracy'])
    val_loss = np.mean(valid_loss[0]['loss'])
    val_acc = np.mean(valid_loss[0]['accuracy'])
    
    train_log['loss'].append(tra_loss)
    train_log['acc'].append(tra_acc)
    val_log['loss'].append(val_loss)
    val_log['acc'].append(val_acc)
    
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        torch.save(encoding_model.state_dict(), encoding_model_save_file)
        print("save model")
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {tra_loss:.3f} | Train Acc: {tra_acc:.3f}')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc:.3f}')


In [ ]:
# 模型测试
encoding_model.load_state_dict(torch.load(encoding_model_save_file))

test_loss = enc_eval(encoding_model, testDataLoader, criterion)

te_loss = np.mean(test_loss[0]['loss'])
te_acc = np.mean(test_loss[0]['accuracy'])

print(f'| Test Loss: {te_loss:.3f} | Test acc: {te_acc:.3f} |')

In [ ]:
a = torch.ones(20,6)
b = torch.randn(20,6)
a,b

## Prediciton

In [ ]:
class GRU_Predict(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        """
        :param input_size:输入维度
        :param hidden_size:隐藏层神经元个数
        
        """
           
        super().__init__()
        
        self.attention = nn.Linear(20*3*6, 20*6)
        
        self.GRU = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        
        self.Linear = nn.Linear(2*hidden_size, 1)
        
        self.Sigmoid = nn.Sigmoid()
        
    def forward(self, s_encoding, m_encoding, l_encoding):
        
        encoding = torch.cat([s_encoding, m_encoding, l_encoding], dim=1).view(len(s_encoding), -1)
        # encoding = [batch_size, short_dim+middle_dim+long_dim=(20*3) * hid_dim=6]
        
        encoding = self.attention(encoding)
        
        encoding = encoding.view(len(encoding), -1, 6)
        
        output, hn = self.GRU(encoding)

        out = output[:, -1, :]
        # out = [batch, num_directions * hidden_size]

        out = self.Linear(out)
        
        out = self.Sigmoid(out)
        
        return out        

In [ ]:
class LSTM_Predict(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        """
        :param input_size:输入维度
        :param hidden_size:隐藏层神经元个数
        
        """
           
        super().__init__()
        
        self.attention = nn.Linear(20*3*6, 20*6)
        
        self.LSTM = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        
        self.Linear = nn.Linear(2*hidden_size, 1)
        
        self.Sigmoid = nn.Sigmoid()
        
    def forward(self, s_encoding, m_encoding, l_encoding):
        
        #  encoding = (batch, seq_len, input_size)
        # encoing = [batch_size, short_dim/middle_dim/long_dim=(20), hid_dim=6]
        
        encoding = torch.cat([s_encoding, m_encoding, l_encoding], dim=1).view(len(s_encoding), -1)
        # encoding = [batch_size, short_dim+middle_dim+long_dim=(20*3) * hid_dim=6]
        
        encoding = self.attention(encoding)
        
        encoding = encoding.view(len(encoding), -1, 6)
        
        output, hn = self.LSTM(encoding)
        
        # output =  (batch, seq_len, num_directions * hidden_size)
        # hn =  (batch, num_layers * num_directions, hidden_size)
        
        out = output[:, -1, :]
        # out = [batch, num_directions * hidden_size]
        
        
        out = self.Linear(out)
        
        out = self.Sigmoid(out)
        
        return out        

In [ ]:
f = '/home/ruyao/self_supervised_model/data/daily_feature-2.pkl'
daily_df = pd.read_pickle(f)

In [ ]:
split_date = (datetime.date(2019,1,1), datetime.date(2020,1,1), datetime.date(2021,1,1), datetime.date(2021,10,1))
pre_train_data = daily_df[(daily_df['dt']>=split_date[0]) & (daily_df['dt']<split_date[1])].reset_index(drop=True)
pre_valid_data = daily_df[(daily_df['dt']>=split_date[1]) & (daily_df['dt']<split_date[2])].reset_index(drop=True)
pre_test_data = daily_df[(daily_df['dt']>=split_date[2]) & (daily_df['dt']<split_date[3])].reset_index(drop=True)
daily_df.shape, pre_train_data.shape, pre_valid_data.shape, pre_test_data.shape

In [ ]:
def build_data_loader_regression(data, batch_size, shuffle):

    X = torch.tensor(data['daily20_features']).to(torch.float32)

    y1 = torch.tensor(data['close_rtn_rank']).to(torch.float32)
    y2 = torch.tensor(data['SR5_rank']).to(torch.float32)
    y3 = torch.tensor(data['SR20_rank']).to(torch.float32)
    y4 = torch.tensor(data['5_day_rtn_rank']).to(torch.float32)
    y5 = torch.tensor(data['20_day_rtn_rank']).to(torch.float32)
    
    data_set = TensorDataset(X, y1, y2, y3, y4, y5) 
    data_loader = DataLoader(data_set, batch_size, shuffle=shuffle, drop_last=False)
    
    return data_loader

In [ ]:
print('start train data..', datetime.datetime.now())
trainLoader = build_data_loader_regression(pre_train_data, 256, True)

print('start valid data..', datetime.datetime.now())
validLoader = build_data_loader_regression(pre_valid_data, 256, True)

print('start test data..', datetime.datetime.now())
testLoader = build_data_loader_regression(pre_test_data, 256, False)
print('dataload end', datetime.datetime.now())

In [ ]:
def pre_train(model, encoding_model, iterator, optimizer, criterion):
    
    model.train()

    epoch_log = {"loss":[], "accuracy":[], "f1_score":[], "recall_score":[]}

    encoding_model.eval()

    for i, batch in tqdm(enumerate(iterator)):

        X, y1, y2, y3, y4, y5 = batch[0], batch[1], batch[2], batch[3], batch[4], batch[5]

        optimizer.zero_grad()

        s_encoding, m_encoding, l_encoding = encoding_model.encoder(X.to(device))

        score = model(s_encoding, m_encoding, l_encoding)           

        loss = criterion(score, y1.view(-1, 1).to(device))    

        loss.backward()

        optimizer.step()

        epoch_log["loss"].append(loss.item())

    return epoch_log

In [ ]:
def pre_eval(model, encoding_model, iterator, criterion, result=False):
    
    model.train()

    epoch_log = {"loss":[], "accuracy":[], "f1_score":[], "recall_score":[]}

    encoding_model.eval()
    
    score_numpy = np.array([])
    
    with torch.no_grad():

        for i, batch in tqdm(enumerate(iterator)):

            X, y1, y2, y3, y4, y5 = batch[0], batch[1], batch[2], batch[3], batch[4], batch[5]

            s_encoding, m_encoding, l_encoding = encoding_model.encoder(X.to(device))

            score = model(s_encoding, m_encoding, l_encoding)      

            score_numpy = np.append(score_numpy, score.detach().cpu().numpy())

            loss = criterion(score, y1.view(-1, 1).to(device))    

            epoch_log["loss"].append(loss.item())

    if result==True:
        return epoch_log, score_numpy
    else:
        return epoch_log

In [ ]:
encoding_model_load_file = encoding_model_save_file
prediction_model_save_file = '/home/ruyao/self_supervised_model/model/prediction_lr0.0001_epoch25_rankrg_closertn-encoding_lr0.0001-2.pt'
pic_name_pre = '/home/ruyao/self_supervised_model/model/prediction_lr0.0001_epoch25_rankrg_closertn-encoding_lr0.0001-2'

In [ ]:
input_size = 6  # 表征维度
hidden_size = 30  # 隐层神经元个数
num_layers = 2  #隐层层数
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

prediction_model = GRU_Predict(input_size, hidden_size, num_layers).to(device)

LEARNING_RATE = 0.0001


criterion_predict = torch.nn.MSELoss()
optimizer_predict = torch.optim.Adam(prediction_model.parameters(), lr = LEARNING_RATE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
N_EPOCHS = 25

best_valid_loss = float('inf')

train_log = {'loss':[], "acc":[], "f1":[], "recall":[]}
val_log = {'loss':[], "acc":[], "f1":[], "recall":[]}

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    train_loss = pre_train(prediction_model, encoding_model, trainLoader, optimizer_predict, criterion_predict)
    valid_loss = pre_eval(prediction_model, encoding_model, validLoader, criterion_predict)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)    
    
    tra_loss = np.mean(train_loss['loss'])
    val_loss = np.mean(valid_loss['loss'])
    train_log['loss'].append(tra_loss)
    val_log['loss'].append(val_loss)
    
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        torch.save(prediction_model.state_dict(), prediction_model_save_file)
        print("save model")
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {tra_loss:.3f} ')
    print(f'\t Val. Loss: {val_loss:.3f} ')
                                                                               

In [ ]:
x = np.arange(1,N_EPOCHS+1) 
y1 =  train_log['loss']
y2 = val_log['loss']
plt.title("loss") 
plt.xlabel("Epochs") 
plt.plot(x,y1, label="train loss") 
plt.plot(x,y2, label="val loss") 
plt.legend()
plt.savefig(pic_name_pre + '_loss.jpg')
plt.show()

In [ ]:
# load已经训练好的参数
encoding_model_save_file = "/home/ruyao/self_supervised_model/model/prediction_lr0.0001_epoch25_rankrg_closertn-encoding_lr0.0001.pt"
prediction_model.load_state_dict(torch.load(encoding_model_save_file))


In [ ]:
validLoader = build_data_loader_regression(pre_valid_data, 256, False)
test_val_loss, test_val_score_np = pre_eval(prediction_model, encoding_model, validLoader, criterion_predict, True)

In [ ]:
trainLoader = build_data_loader_regression(pre_train_data, 256, False)
train_loss, train_score_np = pre_eval(prediction_model, encoding_model, trainLoader, criterion_predict, True)

In [ ]:
# 模型测试

test_loss, score_np = pre_eval(prediction_model, encoding_model, testLoader, criterion_predict, True)

te_loss = np.mean(test_loss['loss'])

print(f'| Test Loss: {te_loss:.3f} ')
# print(f'| Test Loss: {te_loss:.3f} | Test acc: {te_acc:.3f} | Test f1: {te_f1:.3f}| Test recall: {te_recall:.3f}')

In [ ]:
np.sum(test_loss['loss']+test_val_loss['loss'])

In [ ]:
pre_test_data['predict_score'] = score_np.reshape(-1, 1)
df = pre_test_data[['stock_code', "dt", "predict_score"]]
df.columns = ['stock_code', "dt", "score"]
df

In [ ]:
pre_valid_data['predict_score'] = test_val_score_np.reshape(-1, 1)
df2 = pre_valid_data[['stock_code', "dt", "predict_score"]]
df2.columns = ['stock_code', "dt", "score"]
df2

In [ ]:
pre_train_data['predict_score'] = train_score_np.reshape(-1, 1)
df3 = pre_train_data[['stock_code', "dt", "predict_score"]]
df3.columns = ['stock_code', "dt", "score"]
df3

In [ ]:
result_df = pd.concat([df3, df2, df], axis=0)
result_df

In [ ]:
result_root_dir = "/home/ruyao/self_supervised_model/result/prediction_lr0.0001_epoch25_rankrg_closertn-encoding_lr0.0001-2019_2021"
if not os.path.exists(result_root_dir):
    os.mkdir(result_root_dir)
if not os.path.exists(result_root_dir+"/prediction"):
    os.mkdir(result_root_dir+"/prediction")
    
for key, val in result_df.groupby("dt"):
    val.to_csv(f"{result_root_dir}/prediction/{key}.csv", index = False)
    print(f"{result_root_dir}/{key}.csv生成！")
